<a href="https://colab.research.google.com/github/HongyuGu/2021/blob/master/data_processing_for_genderize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# data processing for genderize

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install gender-guesser

     |████████████████████████████████| 379 kB 19.8 MB/s 


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import spacy, nltk, sklearn
import numpy as np
from nltk import wordpunct_tokenize

nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))
from textblob import TextBlob
import dateutil.parser
import gender_guesser.detector as gender

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
nlp = spacy.load('en')

In [5]:
sample = pd.read_csv("/content/drive/MyDrive/Colab_bbear/2015_bbear_sample_sentence_0_1000000.csv", sep=',')

In [6]:
selected_columns = sample[['quotation', 'speaker', 'date']]
selected_columns[['anxiety_feature', 'year', 'month', 'day']] = '0'
for row in range(len(selected_columns)):
  dt = dateutil.parser.isoparse(selected_columns.date[row])
  selected_columns.xs(row)['year'] = dt.strftime('%Y')
  selected_columns.xs(row)['month'] = dt.strftime('%m')
  selected_columns.xs(row)['day'] = dt.strftime('%d')

selected_columns

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:659: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[k] = np.nan
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pa

,quotation,speaker,date,anxiety_feature,year,month,day
0,... a great day for veterans here in Littleton...,Jeanne Shaheen,2015-08-31 02:10:00,0,2015,08,31
1,How FFA scored 32 own goals in 18 months and C...,None,2015-12-08 00:00:00,0,2015,12,08
2,[ Amy ] was placed under an unacceptable amoun...,None,2015-09-10 21:18:59,0,2015,09,10
3,How High Will These Numbers Go?,None,2015-07-23 14:57:49,0,2015,07,23
4,[ Ban ] recalls that hospitals and medical per...,Ban Ki-moon,2015-10-04 08:27:38,0,2015,10,04
...,...,...,...,...,...,...,...
999995,I'm definitely not trying to bad-mouth the Blu...,Randy Wolf,2015-08-19 02:23:07,0,2015,08,19
999996,A lot of that can only come from getting live ...,William Worth,2015-04-16 19:03:00,0,2015,04,16
999997,I'm delighted that I was able to get 12 games ...,Freddie Woodman,2015-10-14 12:35:26,0,2015,10,14
999998,A lot of the hard work has been done in the le...,Anne Field,2015-09-30 18:00:00,0,2015,09,30


In [7]:
# use TextBlob to explore the sentiment of the quotations
lst_polarity = []
lst_subjectivity = []

for i in range(len(selected_columns.quotation)):
    blob = TextBlob(selected_columns.quotation[i])
    sent = blob.sentiment
    lst_polarity.append(sent[0])
    lst_subjectivity.append(sent[1])

selected_columns['polarity'] = lst_polarity
selected_columns['subjectivity'] = lst_subjectivity

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [8]:
# Define a seed list of anxious depression related words
weather = ['rain', 'earthquake']
anxiety_lexicon_base = ['fat', 'bad', 'weak', 'problem', 'tired', 'illusion', 'restless', 'bored', 'crap', 'shit',
                        'fuck', 'sad', 'escape', 'useless', 'meaningless', 'crying', 'reject', 'suffer', 'sleepless', 'never', 
                        'bored', 'afraid', 'unhappy', 'ugly', 'upset', 'awful', 'torture', 'unsuccessful', 'helpless', 'suffer', 
                        'fail', 'sorrow', 'nobody', 'blame', 'damaged', 'shatter', 'pathetic', 'insomnia', 'kill', 'panic', 'lonely', 
                        'hate', 'depressed', 'frustrated', 'loser', 'suicidal', 'hurt', 'painful', 'disappoint', 'broke', 'abandon',
                        'worthless', 'regret', 'dissatisfied', 'lost', 'empty', 'destroy', 'ruin', 'die', 'sick']

In [9]:
count = 0
for i in range(len(selected_columns.quotation)):
  # Casefolding: convert all in lowercase 
  sentence = selected_columns.quotation[i].lower()

  # Tokenize the sentence 
  tokens = wordpunct_tokenize(sentence)

  # Remove the stopword
  filtered_tokens = [word for word in tokens if not word in stopwords]

  # Filter out tokens no in alphabets or numbers 
  filtered_tokens2 = [word for word in filtered_tokens if word.isalnum()] 

  # Check if anxious depression related words are included in a text
  flag = [word for word in filtered_tokens2 if word in anxiety_lexicon_base]
  
  if (len(flag) != 0):
    print(i, flag, '\n')
    selected_columns.anxiety_feature[i] = '1'
    count += 1

print(count)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.

961738 ['never'] 

961766 ['problem'] 

961789 ['die'] 

961799 ['lost'] 

961860 ['fuck', 'hate'] 

961863 ['dissatisfied', 'upset'] 

961865 ['blame'] 

961888 ['broke'] 

961909 ['bad'] 

961919 ['lost'] 

961964 ['problem'] 

961976 ['kill'] 

961981 ['sick'] 

961987 ['lost'] 

962001 ['sad'] 

962006 ['upset'] 

962008 ['problem'] 

962018 ['fat'] 

962020 ['frustrated'] 

962063 ['never'] 

962075 ['problem'] 

962130 ['never'] 

962147 ['problem'] 

962152 ['never'] 

962249 ['bad'] 

962274 ['never'] 

962280 ['never'] 

962289 ['fat'] 

962310 ['never'] 

962312 ['never'] 

962340 ['problem'] 

962363 ['lost'] 

962365 ['bad'] 

962382 ['hurt'] 

962387 ['kill'] 

962421 ['hurt'] 

962426 ['never'] 

962428 ['never'] 

962450 ['never'] 

962451 ['lost'] 

962483 ['never'] 

962485 ['never'] 

962486 ['never'] 

962487 ['never'] 

962489 ['never'] 

962491 ['never'] 

962492 ['lost'] 

962493 ['never'] 

96249

In [10]:
selected_columns['firstname']= '0'


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [11]:
# extract first names of the speakers
# Errors are made here, cuz we just extract the first part of the string. Sometime it can be lastname or a salutation e.g. President


for i in range(len(selected_columns.speaker)):
    if  selected_columns.speaker[i]!= 'None':
        selected_columns.firstname[i] = selected_columns.speaker[i].split(' ')[0]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [12]:
#detect the gender using firstname with gender guesser
selected_columns['gender']= '0'
d = gender.Detector()
for i in range(len(selected_columns.firstname)):
    selected_columns['gender'][i]= d.get_gender(selected_columns.firstname[i])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [13]:
selected_columns.to_csv('selected_columns_2015.csv')